In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score
import xgboost as xgb
from sklearn.decomposition import KernelPCA
import numpy as np

file_path = 'SDSS_DR18_GALAXY.csv'
data = pd.read_csv(file_path).head(60000)

X = data[['u', 'g', 'r', 'i', 'z']]
y = data['redshift']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

kpca = KernelPCA(kernel='rbf', n_components=3) 
X_train_kpca = kpca.fit_transform(X_train)
X_test_kpca = kpca.transform(X_test)

xg_reg = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=50, random_state=42)
xg_reg.fit(X_train_kpca, y_train)
y_pred_xgb = xg_reg.predict(X_test_kpca)

svr_reg = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svr_reg.fit(X_train_kpca, y_train)
y_pred_svr = svr_reg.predict(X_test_kpca)

y_pred_combined = (y_pred_xgb + y_pred_svr) / 2

r2_xgb = r2_score(y_test, y_pred_xgb)
r2_svr = r2_score(y_test, y_pred_svr)
r2_combined = r2_score(y_test, y_pred_combined)

print(f"XGBoost R² score: {r2_xgb}")
print(f"SVR R² score: {r2_svr}")
print(f"Combined Model R² score: {r2_combined}")
